In [1]:
import numpy as np
import pandas as pd
import re

C:\Users\USER\AppData\Local\Temp\ipykernel_5216\1137696326.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [5]:
try:
    df_prod = pd.read_csv('Техрежим доб.CSV', delimiter=';', encoding='utf-8', low_memory=False)
except UnicodeDecodeError:
    df_prod = pd.read_csv('Техрежим доб.CSV', delimiter=';', encoding='ANSI', low_memory=False)
try:
    df_mer = pd.read_csv('МЭР.CSV', delimiter=';', encoding='utf-8', low_memory=False)
except UnicodeDecodeError:
    df_mer = pd.read_csv('МЭР.CSV', delimiter=';', encoding='ANSI', low_memory=False)

In [6]:
df_prod.shape, df_mer.shape

((194922, 66), (963654, 24))

### Приводим дату в конвертируемый формат во фрейме с тех.режимом добывающих скважин

In [7]:
df_prod['Дата'] = df_prod['Дата'].str.split('.')
df_prod['Дата'].map(lambda x: x.insert(0, '01'))

def convert_month_to_digit_mode(date_as_list):
    result = date_as_list
    dict_of_date = {'янв': '01', 'фев': '02', 'мар': '03', 'апр': '04', 'май': '05', 'июн': '06', 'июл': '07', 'авг': '08', 'сен': '09', 'окт': '10', 'ноя': '11', 'дек': '12'}
    result[1] = dict_of_date[result[1]]


df_prod['Дата'].apply(convert_month_to_digit_mode)
df_prod['Дата'] = df_prod['Дата'].map(lambda x: '.'.join(x))
# df_prod['Дата'] = df_prod['Дата'].apply(lambda x: pd.to_datetime(x))

### Переименуем необходимые колонки, чтобы названия были одинаковы для обоих сетов

In [8]:
df_mer = df_mer.rename(columns={'имя скважины': 'Скважина', 'дата(дд.мм.гггг)': 'Дата', 'время работы': 'Время работы, ч', 'состояние': 'Состояние', 'пласт': 'Пласт'})
df_prod = df_prod.rename(columns={'Номер скважины': 'Скважина', 'Состояние на конец месяца': 'Состояние'})

In [9]:
df_mer = df_mer.sort_values(by=['Дата'], ascending=False).reset_index(drop=True)
df_prod = df_prod.sort_values(by=['Дата'], ascending=False).reset_index(drop=True)

### Делаем слияние по пересекающимся значениям колонок "Скважина" и "Дата"

In [10]:
df_prod = df_prod.merge(df_mer, how='inner', left_on=['Скважина', 'Дата', 'Пласт'], right_on=['Скважина', 'Дата', 'Пласт'])

### Заполняем пропущенные значения в колонках "Время работы, ч"

In [11]:
df_prod['Время работы, ч_x'] = df_prod['Время работы, ч_x'].fillna(df_prod['Время работы, ч_y'])
# df_prod['Qн, т/сут_x'] = df_prod['Qн, т/сут_x'].fillna(df_prod['Qн, т/сут_y'])
# df_prod['Qж, м3/сут'] = df_prod['Qж, м3/сут'].fillna(df_prod['Жидкость, м3'] / (df_prod['Время работы, ч_x'] / 24))

### Приводим выходной файл тех.режима по добыче в итоговый для загрузки вид

In [12]:
df_prod = df_prod.rename(columns={'Скважина': 'Номер скважины', 'Состояние_x': 'Состояние на конец месяца', 'Время работы, ч_x': 'Время работы, ч'})
list_of_columns = ['Номер скважины', 'Куст', 'Дата', 'Состояние на конец месяца', 'Способ эксплуатации', 'Рентабельность', 'Внутренний диаметр эксплуатационной колоны, мм', 'Пласт', 'Qн, т/сут', 'Qж, м3/сут',
                   'Обводненность (объемная), %', 'Рзаб, атм', 'Pпл, атм', 'Коэф. продуктивности, м3/сут/атм', 'KH, мД м', 'Скин-фактор', 'Радиус контура питания, м', 'Динамический уровень, м', 'Буферное давление, атм',
                   'Pлин, атм', 'Pзатр, атм', 'Давление на приеме насоса, атм', 'Статический уровень', 'Рзатр при Нстат, атм', 'Тип насоса', 'Дата изм. параметров насоса', 'Глубина спуска насоса, м',
                   'Номинальный напор ЭЦН, м', 'Частота работы ЭЦН, Гц', 'Сила тока ЭЦН, А', 'Номинальная производительность, м3/сут', 'Тип пакера', 'Дата установки пакера', 'Глубина установки пакера, м',
                   'Диаметр штуцера, мм', 'В-сть нефти в пластовых условиях, сПз', 'Плотность нефти (агента закачки для нагнетательных) в поверхностных условиях', 'Объемный коэффициент нефти, м3/м3',
                   'Замеренный газовый фактор, м3/т', 'Глубина верхних дыр перфорации, м', 'Удлинение, м', 'Перфорированная мощность, м', 'Нефтенасыщенная (для добывающих) / эффективная (для нагнетательных) толщина, м',
                   'Внешний диаметр НКТ, мм', 'Вязкость жидкости (агента закачки для нагнетательных) в поверхностных условиях, сПз', 'Добыча нефти, т', 'Добыча жидкости/закачка агента для нагнетательных, м3',
                   'Время работы, ч', 'Время работы в периодическом режиме / под циклической закачкой, ч', 'Дебит нефти потенциальный (технологический), т/сут', 'Дебит жидкости потенциальный (технологический), м3/сут',
                   'Плотность воды в пластовых условиях, г/см3', 'Qж с поправкой на диаметр эксп. колонны (технологический), м3/сут', 'Пуск', 'Остановка', 'Проницаемость', 'Тип', 'Примечание']

In [13]:
df_prod = df_prod[list_of_columns]

### Переименовываем объекты разработки в тех.режиме по добывающим и нагнетательным скважинам

In [14]:
df_inj = pd.read_csv('Техрежим наг.CSV', delimiter=';', encoding='utf-8', low_memory=False)

### Приводим дату в конвертируемый формат во фрейме с тех.режимом нагнетательных скважин

In [15]:
df_inj['Дата'] = df_inj['Дата'].str.split('.')
df_inj['Дата'].map(lambda x: x.insert(0, '01'))

df_inj['Дата'].apply(convert_month_to_digit_mode)
df_inj['Дата'] = df_inj['Дата'].map(lambda x: '.'.join(x))
# df_inj['Дата'] = df_inj['Дата'].apply(lambda x: pd.to_datetime(x))

### Переименовываем объекты разработки

In [16]:
list_of_objects_of_field = {'АВ1/3': 'АВ1_3', 'ЮВ1/1': 'ЮВ1_1'}

In [17]:
df_prod['Пласт'] = df_prod['Пласт'].replace(list_of_objects_of_field)
df_inj['Пласт'] = df_inj['Пласт'].replace(list_of_objects_of_field)

In [18]:
# df_prod.to_excel('Техрежим доб.xlsx', index=False)
# df_inj.to_excel('Техрежим наг.xlsx', index=False)

In [19]:
df_prod = df_prod.drop_duplicates()
df_inj = df_inj.drop_duplicates()

In [20]:
df_prod.to_csv('Техрежим доб.csv', encoding='mbcs', sep=';', index=False, decimal=',')
df_inj.to_csv('Техрежим наг.csv', encoding='mbcs', sep=';', index=False, decimal=',')

In [21]:
df_prod.shape, df_mer.shape

((179151, 58), (963654, 24))

In [25]:
df_mer.drop_duplicates()

,Скважина,Дата,Пласт,характер работы,Состояние,способ эксплуатации,причина простоя,"Время работы, ч",время сбора,время простоя,...,конденсат,агент доб.,V нефти,V воды,V других жидк.,газ.шапка,агент зак.,закачка,техн.зак.,пот.воды
0,SPK833Р,01.12.2023,АВ1/3,НЕФ,ЛИК,Без способа,NaN,0,"0,00",0,...,"0,00",-1,"0,00","0,00","0,00","0,00",-1,"0,00","0,00","0,00"
1,4769,01.12.2023,АВ2,НАГ,Б/Д ТГ,NaN,NaN,0,"0,00",0,...,"0,00",-1,"0,00","0,00","0,00","0,00",-1,"0,00","0,00","0,00"
2,130Р,01.12.2023,АВ1/3,НЕФ,ПЬЕЗ,ЭЦН,NaN,0,"0,00",0,...,"0,00",-1,"0,00","0,00","0,00","0,00",-1,"0,00","0,00","0,00"
3,111,01.12.2023,АВ2,НЕФ,КОНС,ЭЦН,NaN,0,"0,00",0,...,"0,00",-1,"0,00","0,00","0,00","0,00",-1,"0,00","0,00","0,00"
4,794,01.12.2023,АВ1/3,НАГ,РАБ.,NaN,NaN,744,"0,00",0,...,"0,00",-1,"0,00","0,00","0,00","0,00",-1,"0,00","0,00","0,00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
963649,155Р,01.01.1967,АВ1/3,НЕФ,ОСВ ПР ЛЕТ,ФОН,NaN,0,"0,00",0,...,"0,00",-1,"0,00","0,00","0,00","0,00",-1,"0,00","0,00","0,00"
963650,120,01.01.1967,БВ8,НЕФ,ОСВ ПР ЛЕТ,ФОН,NaN,0,"0,00",0,...,"0,00",-1,"0,00","0,00","0,00","0,00",-1,"0,00","0,00","0,00"
963651,124Р,01.01.1967,БВ8,НЕФ,ОСВ ПР ЛЕТ,ФОН,NaN,0,"0,00",0,...,"0,00",-1,"0,00","0,00","0,00","0,00",-1,"0,00","0,00","0,00"
963652,128Р,01.01.1966,БВ8,НЕФ,ОСВ ПР ЛЕТ,ФОН,NaN,0,"0,00",0,...,"0,00",-1,"0,00","0,00","0,00","0,00",-1,"0,00","0,00","0,00"
